# Tensorflow & Named-entity recognition tutorial

09/04/2018

Никишина Ирина (НИУ ВШЭ, ИСП РАН)

в тьюториале содержатся материалы лекций Е. Черняк (https://github.com/echernyak/ML-for-compling)

Requirements (из тех, которые явные и про которые я помню):

* tensorflow
* tqdm
* nltk
* pymorphy2

## Intro

### Что такое нейронные сети и с чем их едят (кажется, вы все уже все знаете)

### Сеть прямого распространения  для классификации текстов


![title](img/mlp.png)

* $x$ - входное векторное представление текста
* $h$ – скрытые слои с нелинейными функциями активации
* $y$ – выходы, как правило, один $y$ соответствует одной метке класса 

$NN_{MLP2}(x) = y$

$h_1 = g^1(xW^1 + b^1)$

$h_2 = g^2(h^1 W^2 + b^2)$

$y = h^2 W^3$

### Нелинейные функции активации

![title](img/activation.png)

### dropout-регуляризация

$NN_{MLP2}(x) = y$

$h_1 = g^1(xW^1 + b^1)$

$m^1 ~ Bernouli(r^1)$

$\hat{h^1} = m^1 \odot h^1$

$h_2 = g^2(\hat{h^1} W^2 + b^2)$

$m^2 ~ Bernouli(r^2)$

$\hat{h^2} = m^2 \odot h^2$

$y =\hat{h^2} W^3$

### Векторное представление текста 
* BOW (bag of words) – разреженное (sparse) векторное представление текста
* CBOW (continious bag of words)  – плотное (dense) векторное представление текста

$w_i$ – слово, $d_{emb}$ – размерность эмбеддинга слова, $E_{[w_i]}$ = $\textbf{w}_i$


#### Padding
Входные тексты имеют переменную длинну, что неудобно, поэтому предположим, что они все состоят из одинакового количества слов, только часть из этих слов – баластные символы pad/


#### Неизвестные слова (OOV)
Если в тестовом множестве встретилось неизвестное слово, то можно 
* заменить его на pad;
* заменить его на unk.  Однако в обучающем множестве unk никогда не встречается, поэтому его нужно добавить в обучающее множество искусственным образом. 


#### Word dropout - регуляризация 
Заменяем каждое слово на unk с вероятностью $\frac{\alpha}{|V| + \alpha}$


## Собственно, задача

### FactRuEval2016

* devset   (122 texts,  ≈ 30940 tokens) 
* testset  (132 texts,  ≈ 59382 tokens)  

file structure:

![title](img/s1.PNG) 

![title](img/s2.PNG) 

![title](img/s3.PNG)

В рамках тестового задания в ИСП РАН была раелизована noname система, позволяющая обрабатывать данные FactRuEval и производить sequence labeling используя различные модели машинного обучения (SVM, Multilayer perceptron, LSTM, biLSTM, CNN) 

In [1]:
import tensorflow as tf
import main
from machine_learning.rnn import RNN
from machine_learning.cnn import CNN
from machine_learning.rnn_trainer import RNNTrainer
from machine_learning.cnn_trainer import CNNTrainer
from machine_learning.svm_model_trainer import SvmModelTrainer

Model_path -- путь к обученной fasstext модели

window -- размер окна (для учета контекста)

ngram_affixes -- размер аффиксов (длина префиксов и суффиксов)

In [2]:
trainset_path = "../data/devset"
testset_path = "../data/testset"
output_path = "../output"
model_path = "../data/ru.bin"
window=2
ngram_affixes=2

In [3]:
morph_analyzer = main.pymorphy2.MorphAnalyzer()
output_path = main.os.path.join(output_path, main.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
#embedding_model = main.get_model_for_embeddings(model_path)
#print('Model is ready')
train_documents = main.get_documents_with_tags_from(trainset_path, morph_analyzer)
print('Docs are ready for training', main.datetime.now())
test_documents = main.get_documents_without_tags_from(testset_path, morph_analyzer)
print('Docs are ready for testing', main.datetime.now())

Docs are ready for training 2018-04-09 07:52:57.032337
Docs are ready for testing 2018-04-09 07:53:28.844042


![title](img/bilou.png)

In [4]:
feature = main.get_composite_feature(window, train_documents, ngram_affixes, None)
print('Features loaded', main.datetime.now())
tags = main.compute_tags()
print("tags = {}".format(tags))

Features loaded 2018-04-09 07:53:34.038209
tags = ['O', 'BPER', 'BLOC', 'BORG', 'IPER', 'ILOC', 'IORG', 'LPER', 'LLOC', 'LORG', 'UPER', 'ULOC', 'UORG']


## Сверточные нейронные сети [Convolutional neural networks, CNN]

* Заимствованы из области компьютерного зрения
* Пик популярности пришелся на 2014 (до +10% аккуратности в задачах классификации), со временем были вытеснены рекуррентными нейронными сетями 
* Помогают справится  с проблемой переменной длины входов (CNN VS window-based NN)

### Слой свертки

#### Фильтр [filter]:
* $w_{1,n}$ – последовательность слов, $k$  – размер окна
* $w_i$ , $d_{emb}$ – размерность эмбеддинга слова,  $\textbf{w}_i \in \mathbb{R}^{d_{emb}} $
* $\textbf{x}_i = [\textbf{w}_{i}, \textbf{w}_{i+1}, \ldots, \textbf{w}_{i+k-1}]$, $\textbf{x}_i \in \mathbb{R}^{k d_{emb}}$

Фильтр: $p_i = g(\textbf{x}_i  u)$, $p_i \in \mathbb{R}$, $u \in \mathbb{R}^{k d_{emb}}$


![title](img/cnn1.png)


Преобразуем каждое входное окно, но пока размерность входа не уменьшается!

#### Слой субдискретизации (пулинга, [pooling])

* $h_i$ – выходные значения фильтра

$\max$-пулинг:	$c = \max_i h_i$


![title](img/cnn2.png)

* Выбираем самый важный признак из полученных на предыдущем шаге 
* Можем использовать и $\min$, и усреднение



#### Многомерные фильтры

Используем $\textit{l}$ разных фильтров: $u_{1}, \ldots, u_{\textit{l}}$: 

$\textbf{x}_i = [\textbf{w}_{i}, \textbf{w}_{i+1}, \ldots, \textbf{w}_{i+k-1}]$

$\textbf{p}_i = g(\textbf{x}_i \cdot  U+b)$

$\textbf{p}_i \in \mathbb{R}^{\textit{l}} $, $\textbf{x}_i \in \mathbb{R}^{k d_{emb}}$, $U \in \mathbb{R}^{k d_{emb} \times \textit{l}}$, $b \in \mathbb{R}^{\textit{l}} $

![title](img/cnn4.png)


$\max$-пулинг:	$c_j = \max_i h_{i,j}, j \in [0,\textit{l}]$


![title](img/cnn6.png)

Kim Y. Convolutional Neural Networks for Sentence Classification. 2014

In [5]:
cnn = CNN(input_size=int(feature.get_vector_size()), output_size=len(tags), hidden_size=100, batch_size=8)
model_trainer = CNNTrainer(epoch=1, nn=cnn, tags=tags)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [ ]:
class CustomCNN():
    def __init__(self, input_size, output_size, hidden_size, batch_size, filter_sizes=(3, 4, 5)):
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.filter_sizes = filter_sizes

        #self.x = 
        #self.y = 
        #self.seqlen = 

        output_layer = self.__add_convolution_layer(self.x, self.input_size, self.hidden_size)

        self.outputs = tf.layers.dense() #???

        self.loss = self.__add_loss(self.outputs)
        self.train = self.__add_update_step(self.loss)

        self.init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(self.init)

In [ ]:
def __add_convolution_layer(self, input_tensor, input_size, output_size):
    input_tensor_expand = tf.expand_dims(input_tensor, -1)
    conv_outputs = []

    for i in range(len(self.filter_sizes)):
        filter_size = self.filter_sizes[i]
        conv_1 = self.__apply_convolution(filter_size, input_tensor_expand, input_size, output_size)
        conv_2 = self.__apply_convolution(filter_size, input_tensor_expand, input_size, output_size)
        result = conv_1 * tf.sigmoid(conv_2)
        #max_pool_outputs = tf.nn.max_pool(result, [1, filter_size, 1, 1], [1, 1, 1, 1], 'SAME')
        conv_outputs.append(result)

    conv_out = tf.concat(conv_outputs, ?)
    conv_out = tf.squeeze(conv_out, [?])
    return conv_out

In [ ]:
def __apply_convolution(self, filter_size, input_tensor, input_size, output_size):
        filter = tf.Variable(tf.random_normal([?, ?, 1, ?], stddev=0.1))
        bias = tf.Variable(tf.constant(0.1, shape=[?]), name="b")
        conv_outputs = tf.nn.conv2d(?, filter, [1, 1, ?, 1], 'SAME')
        conv_outputs = tf.tanh(tf.nn.bias_add(conv_outputs, bias))
        return conv_outputs

In [ ]:
def __add_loss(self, logits):
        #mask = 

        #losses = 

        #loss = 
        return loss


In [ ]:
def __add_update_step(self, loss):

    #optimizer = 
    #update_step = 
    return update_step

In [ ]:
custom_cnn = CustomCNN(input_size=int(feature.get_vector_size()), output_size=len(tags), hidden_size=100, batch_size=8)
custom_model_trainer = CNNTrainer(epoch=5, nn=custom_cnn, tags=tags)

In [ ]:
main.train_and_compute_nes_from(model_trainer=model_trainer, feature=feature, train_documents=train_documents,
                               test_documents=test_documents, output_path=output_path)
print("Testing finished", main.datetime.now())
print('Output path: \n {}'.format(output_path))

Vectors are created 2018-04-09 07:59:41.942645
30940


loss: 0.00011640788, batch: 195, epoch: 0: 100%|██████████| 196/196 [01:56<00:00,  1.69it/s] 



loss: 0.0002458325  epoch: 0 
Training finished 2018-04-09 08:01:39.283275


### Рекуррентные нейронные языковые модели

RNN позволяют уйти от Марковских допущений и позволяют учитывать предысторию произвольной длины.

$x_{1:n} = x_1, x_2, \ldots, x_n$, $x_i \in \mathbb{R}^{d_{in}}$

$y_n = RNN(x_{1:n})$, $y_n \in \mathbb{R}^{d_{out}}$

Для каждого префикса $x_{i:i}$ $y_i$ – выходной вектор.

$y_i = RNN(x_{1:i})$

$y_{1:n} = RNN^{*}(x_{1:n})$, $y_i \in \mathbb{R}^{d_{out}}$

![rnn](img/rnn1.png)

#### Управляемые архитектуры

RNN трудно обучать: проблема исчезающего градиента. Уйти от нее помогают управляемые нейроны специального вида: LSTM и GRU.

$s_i$ – память нейронной сети. Каждое использование $R$ считывает и видоизменяет всю память. 

Управляемый доступ к памяти: $g \in {0,1}^n$:

$s_{i+1} \leftarrow g \odot x + (1-g) \odot s_{i}$

Дифференцируемое управление:

$g \in \mathbb{R}^n $:

$s_{i+1} \leftarrow \sigma(g) \odot x + (1-g) \odot s_{i}$

http://colah.github.io/posts/2015-08-Understanding-LSTMs/
    

![title1](img/LSTM3-chain.png)

![image.png](img/bilstm.png)
author: A. Sysoev (Wikification encoder)

In [5]:
with tf.variable_scope('scope', reuse = tf.AUTO_REUSE ):
    rnn = RNN(input_size=int(feature.get_vector_size()), output_size=len(tags), hidden_size=100, batch_size=8, bilstm=True)
    bilstm_trainer = RNNTrainer(epoch=100, nn=rnn, tags=tags)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [ ]:
class BiLSTM():
    def __init__(self, input_size, output_size, hidden_size, batch_size, bilstm=False):
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.x = 
        self.y = 
        self.seqlen =
        
        self.fw_cell = tf.contrib.rnn.BasicLSTMCell(?)
        self.bw_cell = tf.contrib.rnn.BasicLSTMCell(hidden_size)
        self.mid_outputs, state = tf.nn.bidirectional_dynamic_rnn(self.fw_cell, self.bw_cell, self.x,
                                                                      sequence_length=self.seqlen,
                                                                      dtype=tf.float32)
        self.mid_outputs = tf.concat(self.mid_outputs, 2)
        
        self.outputs = tf.contrib.layers.fully_connected(self.mid_outputs, size?, activation_fn=?)
        
        self.cross_entropy = 
        self.train = 

        self.init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(self.init)


In [6]:
main.train_and_compute_nes_from(model_trainer=bilstm_trainer, feature=feature, train_documents=train_documents,
                               test_documents=test_documents, output_path=output_path)
print("Testing finished", datetime.now())
print('Output path: \n {}'.format(output_path))

Vectors are created 2018-04-09 03:36:14.813298
30940


loss: 0.25211728, batch: 13, epoch: 0:   7%|▋         | 13/196 [00:23<05:26,  1.79s/it]

KeyboardInterrupt: 

### Спасибо за внимание. Ура!